**Initialization**

In [115]:
import google.generativeai as genai
import json
import os
from dotenv import load_dotenv
import easyocr
import cv2
from rapidfuzz import fuzz

load_dotenv()

True

**Gemini Model**

In [116]:
def generate(prompt, image_path) -> list | dict:
    api_key = os.getenv("GEMINI_API_KEY")
    genai.configure(api_key=api_key)

    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash",
        system_instruction="You are a helpful assistant that extracts newspaper fields from images.",
        generation_config={"response_mime_type": "application/json"}
    )

    with open(image_path, "rb") as image_file:
        image_bytes = image_file.read()

    response = model.generate_content([
        {"text": prompt},
        {"mime_type": "image/png", "data": image_bytes}
    ])

    raw_json = response.text
    data = json.loads(raw_json)

    return data

**Call to generate from Gemini**

In [117]:
headline_schema = {
	"type": "array",
	"items": {"type": "string"}
}

headline_prompt = (
	"You are given a newspaper image. "
	"Extract only the article all possible headlines — ignore advertisements, captions, subheadlines, and any other text. "
	"Return the result strictly matching this JSON schema:\n\n"
	f"{json.dumps(headline_schema, indent=2)}"
)

target_image_path = "page_56.png"

headlines = generate(headline_prompt, target_image_path)

for i, headline in enumerate(headlines):
    headlines[i] = headline
print(headlines)
# Dev log
print(json.dumps(headlines, indent=2, ensure_ascii=False))

['PH share prices dip; peso climbs to 56.65 a dollar', 'Metro Pacific unit acquires coconut processor for P1b', 'Concepcion Industrial booked P355-m net income in second quarter', 'Manila Water aims to expand Project i-Float', 'Ayala Corp., four subsidiaries retain spots on FTSE4Good Index']
[
  "PH share prices dip; peso climbs to 56.65 a dollar",
  "Metro Pacific unit acquires coconut processor for P1b",
  "Concepcion Industrial booked P355-m net income in second quarter",
  "Manila Water aims to expand Project i-Float",
  "Ayala Corp., four subsidiaries retain spots on FTSE4Good Index"
]


**EasyOCR reads**

In [118]:
reader = easyocr.Reader(['en'])
image = cv2.imread(target_image_path)
results = reader.readtext(image)

image_raw = image.copy()
for (top_left, top_right, bottom_right, bottom_left), text, confidence in results:
    tl = (int(top_left[0]), int(top_left[1]))
    br = (int(bottom_right[0]), int(bottom_right[1]))
    cv2.rectangle(image_raw, tl, br, (0, 0, 255), 2)
    coord_label = f"{tl} {br}"
    cv2.putText(image_raw, coord_label, (tl[0], tl[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

output_path_raw = target_image_path.replace(".png", "_ocr_boxes.png")
cv2.imwrite(output_path_raw, image_raw)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


True

**Finds the coordinates of the headings**

In [119]:
bounding_box_text = []
bounding_box_coordinates = []

for coordinates, text, _ in results:
    print(text)
    for headline in headlines:
        
        score = fuzz.partial_ratio(headline, text)

        if score > 80:

            top_left, _, bottom_right, _ = coordinates
            
            current_box = ((int(top_left[0]), int(top_left[1])),
                           (int(bottom_right[0]), int(bottom_right[1])))
            bounding_box_text.append(text)
            bounding_box_coordinates.append(current_box)

print(bounding_box_text)
print(bounding_box_coordinates)
print(len(bounding_box_text), len(bounding_box_coordinates))

for i in range(len(bounding_box_text)):
    
    print(bounding_box_text[i])
    print(bounding_box_coordinates[i])


Bz
FRIDAY, JULY 25,2025
BUSINESS
extrastory2o  @gmail.com
Standard
PH share prices =
peso
climbs to 56.65a dollar
SHARE prices closed
lower
Thursday as investors awaited the
results of ongoing
tariff
negotiations between
the U.S:
and
its
major trading partners_
The 30-company
Philippine Stock Exchange index slipped 18.09
points,
or 0.28 percent; to close at 6,444.14.
The broader all shares
index, however; rose 1.04 points, or 0.03 percent, to 3,808.39.
The peso strengthened to 56.65 to the U.S. dollar Thursday from
56.881 Wednesday:
The local
bourse ignored gains posted by Wall Street overnight.
Most Asian stocks also ended higher Thursday as hopes for tariff
deals lifted investor sentiment.
Michael Ricafort, chief economist at Rizal Commercial Banking
said the market' =
decline was
healthy correction after the
NUTRI-LICIOUS ADVOCACY: (From left) Chef Geo Punsalan, member; Nutrition Council; Del Monte Culinary Solutions Kitchen;
index rose for four consecutive
trading days.
Alvin Manal

In [120]:
def is_close(coordinate1, coordinate2, gap_x=5, gap_y=20):
    (x1_min, y1_min), (x1_max, y1_max) = coordinate1
    (x2_min, y2_min), (x2_max, y2_max) = coordinate2

    # Expand both boxes by the gap
    x1_min, y1_min, x1_max, y1_max = x1_min - gap_x, y1_min - gap_y, x1_max + gap_x, y1_max + gap_y
    x2_min, y2_min, x2_max, y2_max = x2_min - gap_x, y2_min - gap_y, x2_max + gap_x, y2_max + gap_y

    # Overlap condition (after expansion)
    horizontal_overlap = not (x1_max < x2_min or x2_max < x1_min)
    vertical_overlap = not (y1_max < y2_min or y2_max < y1_min)

    return horizontal_overlap and vertical_overlap


new = {}
print("Headlines:", headlines)
for headline in headlines:
    new[headline] = []

for i, text in enumerate(bounding_box_text):
    box = bounding_box_coordinates[i]
    for headline in new:
        score = fuzz.partial_ratio(headline, text)
        if score > 80:
            if not new[headline]:
                new[headline].append({"text": text, "box": box})
                print("new")
                print(f"{text=}, {box=}")
            else:
                for i, currentBox in enumerate(new[headline]):
                    score = fuzz.ratio(new[headline][i]["text"], headline)
                    print(f"{text=}, {box=}, {currentBox=}, {score=}")
                    if score >= 90:
                        break
                    if is_close(currentBox["box"], box):
                        
                        
                        (ex_tl_x, ex_tl_y), (ex_br_x, ex_br_y) = currentBox["box"]
                        (tl_x, tl_y), (br_x, br_y) = box

                        new_tl = (min(ex_tl_x, tl_x), min(ex_tl_y, tl_y))
                        new_br = (max(ex_br_x, br_x), max(ex_br_y, br_y))

                        new[headline][i]["box"] = (new_tl, new_br)
                        new[headline][i]["text"] += " " + text
                        break
                else:
                    new[headline].append({"text": text, "box": box})

print(json.dumps(new, indent=2, ensure_ascii=False))

Headlines: ['PH share prices dip; peso climbs to 56.65 a dollar', 'Metro Pacific unit acquires coconut processor for P1b', 'Concepcion Industrial booked P355-m net income in second quarter', 'Manila Water aims to expand Project i-Float', 'Ayala Corp., four subsidiaries retain spots on FTSE4Good Index']
new
text='PH share prices =', box=((101, 671), (771, 852))
text='peso', box=((927, 706), (1144, 838)), currentBox={'text': 'PH share prices =', 'box': ((101, 671), (771, 852))}, score=47.76119402985075
text='climbs to 56.65a dollar', box=((98, 815), (1103, 966)), currentBox={'text': 'PH share prices =', 'box': ((101, 671), (771, 852))}, score=47.76119402985075
new
text='and', box=((1099, 1035), (1173, 1073))
text='at', box=((1188, 1805), (1231, 1843)), currentBox={'text': 'and', 'box': ((1099, 1035), (1173, 1073))}, score=13.043478260869568
new
text='Metro Pacific unit acquires', box=((1303, 1875), (3643, 2181))
text='coconut processor for P1b', box=((1287, 2123), (3558, 2423)), currentB

In [121]:
for key in new:
    print(new[key])
    for i in range(len(new[key])):
        for j in range(i + 1, len(new[key])):
            if is_close(new[key][i]["box"], new[key][j]["box"]):
                print(f"Merging {new[key][i]} and {new[key][j]}")
                (ex_tl_x, ex_tl_y), (ex_br_x, ex_br_y) = new[key][i]["box"]
                (tl_x, tl_y), (br_x, br_y) = new[key][j]["box"]

                new_tl = (min(ex_tl_x, tl_x), min(ex_tl_y, tl_y))
                new_br = (max(ex_br_x, br_x), max(ex_br_y, br_y))

                new[key][i]["box"] = (new_tl, new_br)
                new[key][i]["text"] += " " + new[key][j]["text"]
                del new[key][j]
                break
print(new)

[{'text': 'PH share prices = climbs to 56.65a dollar dip;', 'box': ((98, 671), (1103, 966))}, {'text': 'peso', 'box': ((927, 706), (1144, 838))}, {'text': 'are', 'box': ((2918, 3134), (2982, 3172))}, {'text': 'to to', 'box': ((2919, 3309), (2965, 3340))}, {'text': 'to to', 'box': ((3150, 5027), (3193, 5065))}, {'text': 'to to', 'box': ((3224, 5159), (3267, 5197))}, {'text': 'to to', 'box': ((3287, 5333), (3330, 5371))}, {'text': 'sha', 'box': ((515, 5425), (559, 5443))}]
Merging {'text': 'PH share prices = climbs to 56.65a dollar dip;', 'box': ((98, 671), (1103, 966))} and {'text': 'peso', 'box': ((927, 706), (1144, 838))}
[{'text': 'Metro Pacific unit acquires coconut processor for P1b', 'box': ((1287, 1875), (3643, 2423))}]
[{'text': 'on on on', 'box': ((525, 2434), (576, 2464))}, {'text': 'in in', 'box': ((703, 2428), (746, 2465))}, {'text': 'on on on', 'box': ((1010, 2434), (1056, 2464))}, {'text': 'in in', 'box': ((1188, 3050), (1231, 3087))}, {'text': 'in in', 'box': ((3234, 3087

In [122]:
from fuzzywuzzy import process

image_merged = image.copy()

for key in new:
    query = key
    choices = [i["text"] for i in new[key]]

    if not choices:
        continue  # skip if no choices

    # map choice → index
    choices_dict = {c: i for i, c in enumerate(choices)}

    # get best match
    best_match = process.extractOne(query, list(choices_dict.keys()))
    if best_match:
        text, score = best_match
        index = choices_dict[text]

        tl, br = new[key][index]["box"]
        print(text, tl, br)
        cv2.rectangle(image_merged, tl, br, (0, 255, 0), 5)

# save output
output_path_merged = f"{os.path.splitext(target_image_path)[0]}_result{os.path.splitext(target_image_path)[1]}"
cv2.imwrite(output_path_merged, image_merged)


PH share prices = climbs to 56.65a dollar dip; peso (98, 671) (1144, 966)
Metro Pacific unit acquires coconut processor for P1b (1287, 1875) (3643, 2423)
Concepcion Industrial booked P355-m net income in second quarter (103, 3493) (2762, 3656)
Manila Water aims to expand Project i-Float (2983, 3507) (3578, 3886)
Ayala Corp;, four subsidiaries retain spots on FTSE4Good Index Ayala (1302, 5762) (3423, 5952)


True